# Tutorial: Hands on Open Food Fact Data

A small tutorial to get hands on Open Food Facts Data!

## Getting the data

The Open Food Facts data is available in several way:
* through [the API](https://openfoodfacts.github.io/openfoodfacts-server/api/) - but this is **mandatory** to use it only for real scan / direct usage by users (applications)
* through its exports: https://world.openfoodfacts.org/data



### The parquet export

In this tutorial we will use the **parquet** export available on hugginface, because it's very handy.

For commodity we will use duckdb (but you may as well use pandas directly).
So let's install it first.

In [2]:
# duckdb database
!pip install --quiet duckdb==1.4.1.dev125  # note: 1.4.0 has a problem with sqlalchemy driver
# integrate sql directly in jupyterlab
!pip install --quiet jupysql==0.11.1
# jupysql needs duckdb driver
!pip install --quiet duckdb-engine==0.17.0
# pandas for better outputs
!pip install --quiet pandas
# download huggingface data
!pip install --quiet huggingface_hub

# the Open Food Facts SDK
!pip install --quiet openfoodfacts==3.1.0

In [3]:
import duckdb

%load_ext sql

In [4]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Let's open a database to work

In [5]:
import sqlalchemy
import os
engine = sqlalchemy.create_engine(f'duckdb:///{os.getcwd()}/fr_food_ingredients.db')
%sql engine --alias fr_food

In [6]:
%config SqlMagic.named_parameters="disabled"

Maybe we have a huggingface token to avoid rate limiting

In [7]:
import pathlib
import os
token_path = pathlib.Path('hf-token')
if token_path.exists():
    HF_TOKEN = token_path.open().read().strip()
    os.environ["HF_TOKEN"] = HF_TOKEN
    print(f"HF_TOKEN set to *********{HF_TOKEN[-2:]}") 

HF_TOKEN set to *********fB


Now let's get some data. We will download the parquet locally (4Gb !)

In [8]:
from huggingface_hub import hf_hub_download
# 
if not pathlib.Path('food.parquet').exists():
    hf_hub_download(repo_id="openfoodfacts/product-database", repo_type="dataset", filename="food.parquet", token=HF_TOKEN, local_dir=".")

/home/alex/docker/off-data-tutorial/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From this parquet we can make a smaller duckdb database (but we could query the parquet directly).

A good helper is https://wiki.openfoodfacts.org/DuckDB_Cheatsheet

We will get data for France, where we have at least have ingredients.

For that, we will filter on country having France and number ingredients > 0

In [9]:
%%sql fr_food
DROP TABLE IF EXISTS products;
-- seems to be mandatory to download the file
-- SET unsafe_disable_etag_checks = true;
CREATE TABLE products AS
  SELECT
    *
  FROM
    read_parquet ('food.parquet') AS food
  WHERE
    'en:france' IN food.countries_tags
    AND
    food.ingredients_n is not NULL
;

,Success


In [10]:
%%sql
SELECT count(*) from products;

,count_star()
0,339105


### Other sources

You also have other format for the export.
The mongodb dump, or it's JSONL exports are the more complete, but also the most difficult to read.

One popular export is the CSV export, which is very simple to use, but less structured.

## Looking at fields

Now that we have our data, let's look a bit at what we have.
Let's take for example Gerblés biscuits (among the few nutriscore B biscuits) with code `3175680011480`


In [11]:
%%sql
SELECT * from products where code = '3175680011480';

,additives_n,additives_tags,allergens_tags,brands_tags,brands,categories,categories_tags,categories_properties,checkers_tags,ciqual_food_name_tags,...,states_tags,stores_tags,stores,traces_tags,unique_scans_n,unknown_ingredients_n,unknown_nutrients_tags,vitamins_tags,with_non_nutritive_sweeteners,with_sweeteners
0,4,"[en:e322, en:e336, en:e500, en:e503]","[en:gluten, en:sesame-seeds]",[xx:gerble],Gerblé,"Snacks,Snacks sucrés,Biscuits et gâteaux,Biscu...","[en:snacks, en:sweet-snacks, en:biscuits-and-c...","{'ciqual_food_code': None, 'agribalyse_food_co...",[sebleouf],[unknown],...,"[en:checked, en:complete, en:nutrition-facts-c...","[e-leclerc, carrefour, auchan, monoprix, carre...","E.Leclerc,Carrefour,Auchan,Monoprix,carrefour....","[en:eggs, en:lupin, en:milk, en:mustard, en:nu...",829,0,[],"[en:vitamin-e, en:niacin, en:vitamin-b6, en:th...",<NA>,<NA>


In [12]:
biscuits_data = _

As we can see there are too many columns to see (110), let's get the list nonetheless.

In [13]:
print("\n".join(list(biscuits_data.columns)))

additives_n
additives_tags
allergens_tags
brands_tags
brands
categories
categories_tags
categories_properties
checkers_tags
ciqual_food_name_tags
cities_tags
code
compared_to_category
complete
completeness
correctors_tags
countries_tags
created_t
creator
data_quality_errors_tags
data_quality_info_tags
data_quality_warnings_tags
data_sources_tags
ecoscore_data
ecoscore_grade
ecoscore_score
ecoscore_tags
editors
emb_codes_tags
emb_codes
entry_dates_tags
food_groups_tags
generic_name
images
informers_tags
ingredients_analysis_tags
ingredients_from_palm_oil_n
ingredients_n
ingredients_original_tags
ingredients_percent_analysis
ingredients_tags
ingredients_text
ingredients_with_specified_percent_n
ingredients_with_unspecified_percent_n
ingredients_without_ciqual_codes_n
ingredients_without_ciqual_codes
ingredients
known_ingredients_n
labels_tags
labels
lang
languages_tags
last_edit_dates_tags
last_editor
last_image_t
last_modified_by
last_modified_t
last_updated_t
link
main_countries_tags
m

This is a bit overwhelming, isn't it ?

So let's take a look at some of them, the most usefull for many cases.

That said if you need to dig more they are all documented in the [OpenAPI specficication](https://openfoodfacts.github.io/documentation/docs/Product-Opener/v3/products/get-product-by-barcode/).

### Identifying the product

First, some fields help us identify the product:

Code contains the barcode, it's also the unique identifier for the product:
```{code-cell} ipython3
biscuits_data["code"][0]
```

In [16]:
from textwrap import dedent as _d

In [14]:
_desc = [
    ("code", "contains the barcode, it's also the unique identifier for the product"),
    ("product_name", "is the product name in different languages"),
    ("brands", "is the brand names"),
    ("product_quantity", "is the product quantity in pack"),
    ("product_quantity_unit", "is the product quantity unit"),
]
for field_name, description in _desc:
    print(field_name, description, "\n", biscuits_data[field_name][0], "\n")

code contains the barcode, it's also the unique identifier for the product 
 3175680011480 

product_name is the product name in different languages 
 [{'lang': 'main', 'text': 'Sésame'}
 {'lang': 'en', 'text': 'Gerblé - Sesame Cookie, 230g (8.2oz)'}
 {'lang': 'fr', 'text': 'Sésame'}] 

brands is the brand names 
 Gerblé 

product_quantity is the product quantity in pack 
 230.0 

product_quantity_unit is the product quantity unit 
 g 



### Tags fields

There are a lot of fields ending with "_tags", those are fields that are normalized in some way and are the one to priviledge when doing data analysis.

The philosophy of open food facts project is to let user fill data as they find them on pack, and then try to make sense of those data. This is why we generally don't restrict fields input, but try to make sense of them during the analysis process.

Let's see some informations we can get on the product using this kind of fields.



In [18]:
_desc = [
    (
        "categories_tags", 
        """
        are the categories the products belong to.
        Categories are very useful to grab an entire class of product, or for product comparison,
        but also for scores like Nutri-Score and Green-Score which formula may vary based on category.
        """
    ),
    (
        "ingredients_tags",
        """
        gives a list of ingredients known to be present in the product.
        We will see later on a more precise structure, but this can be handy to quickly easily select products.
        """
    ),
    ("brands_tags", "is the list of brands.\n You can have more than one brands in the sense that one maybe a sub-brand of the other."),
    ("labels_tags", "gives all allegations written on the product, within those are controlled labels"),
    ("allergens_tags", "list allergens"),
]
for field_name, description in _desc:
    print(field_name, _d(description), "\n-", "\n- ".join(biscuits_data[field_name][0]), "\n")

categories_tags 
are the categories the products belong to.
Categories are very useful to grab an entire class of product, or for product comparison,
but also for scores like Nutri-Score and Green-Score which formula may vary based on category.
 
- en:snacks
- en:sweet-snacks
- en:biscuits-and-cakes
- en:biscuits-and-crackers
- en:biscuits 

ingredients_tags 
gives a list of ingredients known to be present in the product.
We will see later on a more precise structure, but this can be handy to quickly easily select products.
 
- en:wheat-flour
- en:cereal
- en:flour
- en:wheat
- en:cereal-flour
- en:unrefined-cane-sugar
- en:added-sugar
- en:disaccharide
- en:sugar
- en:cane-sugar
- en:unrefined-sugar
- en:colza-oil
- en:oil-and-fat
- en:vegetable-oil-and-fat
- en:rapeseed-oil
- en:sesame
- en:seed
- en:wheat-germ
- en:whole-wheat-flour
- en:wholemeal-flour
- en:natural-flavouring
- en:flavouring
- en:magnesium
- en:minerals
- en:emulsifier
- en:raising-agent
- en:sea-salt
- en:salt
- e

As you can see all those tags start with a language code, the a column, and then a dashed name.
The language code is mostly english, but it might be `xx` to mean *in all languages* and, for example, we have `fr:triman` because this lable is only found in France (so as no real english translation).

### Taxonomies

To take full advantages of those tags, one might use [taxonomies](https://wiki.openfoodfacts.org/Taxonomies_introduction).

They contain translations and synonyms for the various entries, as well as properties linked to them.

Hopefully we have a [Python SDK](https://github.com/openfoodfacts/openfoodfacts-python/) that [can help on this task](https://github.com/openfoodfacts/openfoodfacts-python/blob/develop/docs/handle_taxonomies.md).

For example taxonomies might help us get a simple display name for the categories and display some properties

In [20]:
from openfoodfacts.taxonomy import get_taxonomy
categories_taxonomy = get_taxonomy("category")

In [38]:
for category in biscuits_data["categories_tags"][0]:
    cat_taxo = categories_taxonomy[category]
    # some entries might not be in taxonomy
    if not cat_taxo:
        print(category, "is not in the taxonomy\n\n")
    else:
        print(category, 
              "\n  fr:", cat_taxo.get_localized_name("fr"),
              f"(or {', '.join(cat_taxo.synonyms["fr"][1:])})" if len(cat_taxo.synonyms["fr"]) > 1 else "", 
              "\n---\n " , "\n  ".join(":".join([pname, repr(pvalue)]) for pname, pvalue in cat_taxo.properties.items()),
              "\n\n",
        ) 

en:snacks 
  fr: Snacks  
---
  wikidata:{'en': 'Q749316'} 


en:sweet-snacks 
  fr: Snacks sucrés (or sucreries) 
---
  incompatible_with:{'en': 'categories:en:salty-snacks'}
  nova:{'en': '3'} 


en:biscuits-and-cakes 
  fr: Biscuits et gâteaux  
---
  description:{'en': 'A biscuit is a flour-based baked food product, which is typically hard, flat and unleavened.'}
  food_groups:{'en': 'en:biscuits-and-cakes'}
  pnns_group_2:{'en': 'Biscuits and cakes'} 


en:biscuits-and-crackers 
  fr: Biscuits sucrés & biscuits apéritifs  
---
  intake24_category_code:{'en': 'BSCT'} 


en:biscuits 
  fr: Biscuits (or biscuit, biscuits sucrés, biscuit sucré) 
---
  carbon_footprint_fr_foodges_ingredient:{'fr': 'Biscuit au beurre'}
  ciqual_proxy_food_name:{'fr': 'Biscuit sec, sans précision', 'en': 'Biscuit -cookie-'}
  wikipedia:{'en': 'https://en.wikipedia.org/wiki/Biscuit'}
  agribalyse_proxy_food_code:{'en': '24000'}
  ciqual_proxy_food_code:{'en': '24000'}
  wikidata:{'en': 'Q13270'} 




As you can see we retrieved quite interesting categories:
* a proxy for the the ciqual code
* PNNS (nutrition program in France) code
* intake24 category code
* a proxy to agribalyse code
* wikipedia and wikidata entry
* etc.

Also note that the `_tags` fields might also contains items that are not (yet?) in the taxonomy.

The taxonomy is a DAG. As you can see, in `_tags` fields, we not only have the leaf nodes of the taxonomy, but also parents.
It's easy to get the leaf nodes though:

In [46]:
# Convert the ID to TaxonomyNode objects:
categories_nodes = [categories_taxonomy[tag] for tag in biscuits_data['categories_tags'][0] if tag in categories_taxonomy]
# Let's find the leaf nodes using find_deepest_nodes method:
leaf_categories = categories_taxonomy.find_deepest_nodes(categories_nodes)
print("Leaf category:", "\n".join(l.id for l in leaf_categories))

Leaf category: en:biscuits


You might ask what is the leading principle around building the taxonomy: this is the concerns of the Open Food Facts project. So it's more geared toward consumer experience, twisted some time to enable scores computation.
But this is the right place to link to other onthologies etc.

Contribution to alignment with existing onthologies is more than welcome.

**Exercice:** Can you list properties from labels ?

### Nutrition data

One important topic is of course nutrition.

The `nutriments` (yes, it should have been called *nutrients*) fields contains nutrition information.

It is an array of dictionary containing information for each nutrients.

The fields for each nutriments are:
* name - name of nutrient
* 100g and value - quantity for 100g
* unit - the unit for the quantity
* serving - the quantity per serving

We have an array, because of parquets constraints, but we can easily transform it to a dict, indexing by name

In [62]:
biscuits_nutrients = {nutrient["name"]: nutrient for nutrient in biscuits_data["nutriments"][0]}

Let's list availabel fields for our biscuits

In [63]:
print("\n".join(sorted(biscuits_nutrients.keys())))

carbohydrates
carbon-footprint-from-known-ingredients
energy
energy-kcal
energy-kj
fat
fiber
fruits-vegetables-legumes-estimate-from-ingredients
fruits-vegetables-nuts
fruits-vegetables-nuts-estimate-from-ingredients
magnesium
nova-group
nutrition-score-fr
proteins
salt
saturated-fat
sodium
sugars
vitamin-b1
vitamin-b6
vitamin-b9
vitamin-e
vitamin-pp


And look at some important nutrients for our biscuits

In [67]:
import pandas as pd

df = pd.DataFrame(biscuits_nutrients.values(), columns=["name", "100g", "unit", "serving"])
df[df['name'].isin(["energy-kj", "energy-kcal", "carbohydrates", "fat", "salt", "fiber"])]

,name,100g,unit,serving
1,fat,18.00,g,2.070000
2,fiber,4.60,g,0.529000
12,carbohydrates,64.00,g,7.360000
13,salt,0.38,g,0.043700
14,energy-kcal,467.00,kcal,53.700001
16,energy-kj,1961.00,kJ,226.000000


As you see we have the value per serving, but this later is only available if we have the "serving_quantity" data.
(and vice versa, if the primary value, was the serving value, as in the some countries, like the USA).

Note: the `serving_size` field is a textual field with the quantity, while serving_quantity is numeric, result of analyze of serving_size)

In [72]:
print(biscuits_data["serving_quantity"][0])

11.5


### Ingredients

Another key point for food is ingredients. We can derive a lot of useful information from them.

We have the raw information in `ingredient_text`, with one entry per language, with "main" being the entry for product main language (which is the one which will get analyzed)

In [93]:
next(d for d in biscuits_data["ingredients_text"][0] if d["lang"] == "main")["text"]

"Farine de blé 57%, sucre de canne roux, huile de colza, sésame toasté 10,6%, germe de blé 5,4%, farine complète de blé 5,4%, arôme naturel, magnésium, émulsifiant : lécithines, poudres à lever (tartrates de potassium, carbonates de sodium, carbonates d'ammonium), sel de mer, amidon de blé, vitamines (E, PP, B6, B1, B9)."

But this data is not very easy to work with.

Open Food Facts does an analysis of this data and generates the `ingredients` field which contains different data for each ingredients.

It is an array of dict, but encoded in JSON (because we have (non homogenous) array nesting which is not supported natively).

In [89]:
import json
biscuits_ingredients = json.loads(biscuits_data["ingredients"][0])

In [90]:
biscuits_ingredients[0]

{'percent_max': 57.0,
 'percent_min': 57.0,
 'is_in_taxonomy': 1,
 'percent_estimate': 57.0,
 'vegan': 'yes',
 'id': 'en:wheat-flour',
 'text': 'Farine de blé',
 'vegetarian': 'yes',
 'ciqual_food_code': None,
 'percent': 57.0,
 'from_palm_oil': None,
 'ingredients': None,
 'ecobalyse_code': 'a2e25aca-1f42-4bc8-bc0e-4d7c751775aa',
 'processing': None,
 'labels': None,
 'origins': None,
 'ecobalyse_proxy_code': None,
 'quantity': None,
 'quantity_g': None,
 'ciqual_proxy_food_code': '9410'}

Some interesting fields:

* `is_in_taxonomy`, tells us if we recognized this ingredient in the taxonomy
* `id` is the id in the taxonomy
* `vegan`, `vegetarian`, `from_palm_oil`, `ecobalyse_code` are informations derived from the taxonomy entry
* the percentage, which was indicated, was captured correctly

Let's look at the second one

In [92]:
biscuits_ingredients[1]

{'percent_max': 11.0,
 'percent_min': 10.6,
 'is_in_taxonomy': 1,
 'percent_estimate': 10.8,
 'vegan': 'yes',
 'id': 'en:unrefined-cane-sugar',
 'text': 'sucre de canne roux',
 'vegetarian': 'yes',
 'ciqual_food_code': None,
 'percent': None,
 'from_palm_oil': None,
 'ingredients': None,
 'ecobalyse_code': '8f075c25-9ebf-430c-b41d-51d165c6e0d8',
 'processing': None,
 'labels': None,
 'origins': None,
 'ecobalyse_proxy_code': None,
 'quantity': None,
 'quantity_g': None,
 'ciqual_proxy_food_code': '31016'}

The new thing is that you can see we did a very simple estimate of the percentage of this ingredient (`percent_min` and `percent_max`)

Let's look at the "emulsifi$ant" part of ingredients (removing most fields for convenience)

In [102]:
def simplify_ingredients(ingredient):
    return {name: (value if name != "ingredients" else [simplify_ingredients(v) for v in value] if value else None) for name, value in ingredient.items() if name in ["text", "id", "ingredients"]}

In [103]:
simplify_ingredients(biscuits_ingredients[8])

{'id': 'en:emulsifier',
 'text': 'émulsifiant',
 'ingredients': [{'id': 'en:e322', 'text': 'lécithines', 'ingredients': None}]}

As you can see *lécithines* was spotted as a sub ingredient of "émulsifiant".

Next is even more complex

In [104]:
simplify_ingredients(biscuits_ingredients[9])

{'id': 'en:raising-agent',
 'text': 'poudres à lever',
 'ingredients': [{'id': 'en:e336',
   'text': 'tartrates de potassium',
   'ingredients': None},
  {'id': 'en:e500', 'text': 'carbonates de sodium', 'ingredients': None},
  {'id': 'en:e503', 'text': "carbonates d'ammonium", 'ingredients': None}]}

Here, the raising-agent was clearly decomposed in the various sub components